In [2]:
import numpy as np
import pandas as pd
from sqlalchemy.sql import text

from py.db import engine

In [3]:
with engine.connect() as conn:
    statement = text("""ROLLBACK;""")

    query = "SELECT * FROM melding;"
    df = pd.read_sql(query, conn)    

In [4]:
# for convenience, drop any records that haven't got a closing time
print(len(df))
df.laatste_activiteit = df.laatste_activiteit.astype(str)
df = df[df.laatste_activiteit != 'None']
print(len(df))
print(df.iloc[0].laatste_activiteit)

97025
93412
23:57:03.222000


In [5]:
# print(df.iloc[0].laatste_activiteit)
# print(df.iloc[0].laatste_activiteit.astype(str))

def set_date(col):
    # date_formates = ["21 June, 2018", "12/11/2018 09:15:32", "April-21" ]
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S"] # Can add different date formats to this list to test
    for x in date_formats:
        col = pd.to_datetime(col, errors="ignore", format= f"{x}")

    col = pd.to_datetime(col, errors="coerce") # To remove errors in the columns like strings or numbers
    return col

df.laatste_activiteit = df.laatste_activiteit.apply(lambda col: set_date(col=col))
df.laatste_activiteit = pd.to_datetime(df.datum.astype(str) + " " + df.laatste_activiteit.astype(str)).dt.tz_localize(None)


/tmp/ipykernel_743/1965788748.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.laatste_activiteit = pd.to_datetime(df.datum.astype(str) + " " + df.laatste_activiteit.astype(str)).dt.tz_localize(None)


In [146]:
df.iloc[0]


id                                                                  55
datum                                                       2016-01-14
soort_weg                                                         nweg
wegnummer                                                         N321
zijde                                                            beide
hm_paal                                                            5,5
type_controle                                                    Radar
tijd_van_melden                                               20:38:10
details                           bij esso tankstation door vw bus\r\n
laatste_activiteit                          2016-01-14 23:57:03.222000
locatie                                             51.743073,5.779372
locatie_lat                                                  51.743073
locatie_lon                                                   5.779372
weer_type                                                         None
weer_b

In [6]:
print(str(df.iloc[0].datum))
print(str(df.iloc[0].tijd_van_melden))
    
    
    
df.tijd_van_melden = pd.to_datetime(df.datum.astype(str) + " " + df.tijd_van_melden.astype(str))

print(len(df))

2016-01-14
20:38:10
93412


In [36]:
# print(df.iloc[-1].laatste_activiteit)
# print(str(df.iloc[-1].laatste_activiteit))
# 
# df.laatste_activiteit = pd.to_datetime(df.datum.astype(str) + " " + df.laatste_activiteit.astype(str))

20:50:42.823297
20:50:42.823297


/tmp/ipykernel_50282/2895562283.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.laatste_activiteit = pd.to_datetime(df.datum.astype(str) + " " + df.laatste_activiteit.astype(str))


DateParseError: Unknown datetime string format, unable to parse: 2016-01-11 None, at position 0

In [7]:
df.to_parquet("./data/flitsers.parquet")